<a href="https://colab.research.google.com/github/raymondwcs/learning_bert/blob/master/Finetune_pre_trained_BERT_Model_(MLM%2C_NSP%2C_BertForPretraining).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Source:
https://www.youtube.com/watch?v=IC9FaVPKlYc&list=PLIUOU7oqGTLgQ7tCdDT0ARlRoh1127NSO&index=5

In [1]:
!pip install --quiet transformers
!pip install --quiet jsonlines

     |████████████████████████████████| 2.6 MB 5.6 MB/s 
     |████████████████████████████████| 895 kB 38.3 MB/s 
     |████████████████████████████████| 3.3 MB 33.4 MB/s 
     |████████████████████████████████| 636 kB 34.3 MB/s 


In [2]:
from transformers import BertTokenizer, BertForPreTraining, AdamW
from tqdm import tqdm 
import torch
import json
# import jsonlines

CHECKPOINT = 'bert-base-chinese'
# TRAIN_TEXT_FILENAME = './lihkg_channel_15.jsonl'
TRAIN_TEXT_FILENAME = 'aastocks.txt'
BATCH_SIZE = 4

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForPreTraining.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# text = []
# with jsonlines.open(TRAIN_TEXT_FILENAME) as reader:
#   for obj in reader:
#     text.append(obj['text'])

with open(TRAIN_TEXT_FILENAME, "r") as f:
    text = f.readlines()

print('Loaded {} records from {}'.format(len(text), TRAIN_TEXT_FILENAME))
print(text[:5])

Loaded 25 records from aastocks.txt
['該行認為市場仍在調整針對互聯網行業的預期，但大環境來看，各個細分行業的投資週期及行業趨勢逐步清晰。過去兩月行業估值大幅回落，相信針對主要軟件企業的優惠稅率議題會在業績電話會議中大致解釋清楚，而數據私隱及保障等因素隨著相關法例通過亦已反映在股價上，互聯網企業亦已提出所需措施及改善計劃，尚需釐清的領域則包括直播及內容監管、騎手社保問題、線上購物及社區團購前景、短視頻競爭以及投資者對線上券商的投資氣氛等。\n', '美國8月Markit綜合產出指數初值由紀錄高連續第三個月回落至55.4(前值59.9)創八個月低遠差預期，投資者料聯儲局短期不會急於「收水」，加上輝瑞與BioNTech合作研發新冠疫苗正式獲美國當局批准全面使用，紓緩對美國近期確診病例激增憂慮，商品價格全面上揚，尤其是油價急彈逾半成結束七連跌，美股昨晚在銀行、工業、能源、科技及晶片板塊領航全面續升0.6%-1.55%，納指及標普三連漲續跑贏兼齊再創紀錄高。美股期貨全面續升不足0.2%。\n', '亞太區股市全面向好，但除日經及韓股續升1%及1.4%，分別報27,777及3,133外，其餘升幅均少於1%，其中新加坡海峽時報指數回升0.6%報3,107。吉隆坡綜合指數兩連升後，今早微升不足1點報1,523。滬指及深成繼昨天回升約1.5%-2%後，今早高開不足0.2%，分別報3,482及14,558。\n', '恆指午後在新經濟股升幅擴大主導，升幅擴至543點或2.2%，報25,652，現報25,629，續漲519點或2.1%，成交額1,379億元。此外，恆生科技指數升幅擴至6.4%報6,408。\n', '港股方面，早段表現相當不俗，多隻上周急跌之股份均有強勁反彈，恒指亦一度反彈至25,458點，最多升超過600點。不過午後科技股升勢又再度收窄，其他股份升幅亦有所轉弱，恒指升幅便一度收窄至不足200點。\n']


In [7]:
# bag = [item for sentence in text for item in sentence.split('.') if item != '']

# bag = []
# for line in lines:
#   splitted = line.split('。')
#   if (len(splitted) >= 2):
#     bag.append(splitted[0])
#     bag.append(splitted[1])

# bag_size = len(bag)
# print(bag_size)

50


Create 50/50 NSP training data

In [10]:
import random

sentence_a = []
sentence_b = []
label = []

for paragraph in text:
    sentences = [
        # sentence for sentence in paragraph.split('.') if sentence != ''
        sentence for sentence in paragraph.split('。') if sentence != ''
    ]
    num_sentences = len(sentences)
    if num_sentences > 1:
        start = random.randint(0, num_sentences-2)
        # 50/50 whether is IsNextSentence or NotNextSentence
        if random.random() >= 0.5:
            # this is IsNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(sentences[start+1])
            label.append(0)
        else:
            index = random.randint(0, bag_size-1)
            # this is NotNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(bag[index])
            label.append(1)

In [11]:
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt',
                   max_length=512, truncation=True, padding='max_length')


In [12]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

We can now create our labels tensor for MLM, and the next_sentence_label for NSP.

The next_sentence_label tensor is simply a LongTensor built from our label variable.

In [13]:
inputs['next_sentence_label'] = torch.LongTensor([label]).T


And the labels tensor is simply a clone of the input_ids tensor before masking.


In [14]:
inputs['labels'] = inputs.input_ids.detach().clone()

Now we mask tokens in the input_ids tensor using the 15% probability for MLM - ensuring we don't mask CLS, SEP, or PAD tokens.


In [15]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

And now take the indices of each True value within each vector.


In [16]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )


Then apply these indices to each row in input_ids, assigning each value at these indices a value of 103.


In [17]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [18]:
inputs.keys()


dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'next_sentence_label', 'labels'])

We now have all of our inputs and labels ready, so we can begin setting inputs up to be fed into our model during training. We create a PyTorch dataset from our data.


In [20]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

Initialize our data using the MeditationsDataset class.


In [21]:
dataset = MyDataset(inputs)


And initialize the dataloader, which we'll be using to load our data into the model during training.


In [22]:
loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)


Now we can move onto setting up the training loop. First we setup GPU/CPU usage.


In [23]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

Activate the training mode of our model, and initialize our optimizer (Adam with weighted decay - reduces chance of overfitting).


In [24]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

Now we can move onto the training loop, we'll train for two epochs (change epochs to modify this).


In [26]:
from tqdm.notebook import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        next_sentence_label = batch['next_sentence_label'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        next_sentence_label=next_sentence_label,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
model.save_pretrained("outputs")